In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

In [ ]:
def plot_signature_size_effect(signature_sizes, topologies=["gossip", "grid"], **base_run_kwargs):
    """
    Plot how changing signature size affects SNARK1 aggregation time across different topologies.
    
    Args:
        signature_sizes: List of signature sizes to test (in bytes)
        topologies: List of network topologies to test
        **base_run_kwargs: Base simulation parameters
    """
    results = []
    
    print(f"Testing signature sizes: {signature_sizes}")
    print(f"Topologies: {topologies}")
    print("Running simulations...\n")
    
    for topology in topologies:
        print(f"Testing topology: {topology}")
        
        for sig_size in signature_sizes:
            print(f"  Signature size: {sig_size} bytes")
            
            # Create a modified YAML config with the new signature size
            modified_yaml = beamsim.yaml(f"""
# Simulation Backend Configuration
backend: ns3-direct
                                        
snark1_pull: true
snark1_half_direct: true

# Network Topology Configuration
topology: {topology}

# Whether to shuffle validators from the same group to different routers
shuffle: false

# Seed for reproducible simulation results
random_seed: 42

# Role Assignment Configuration
roles:
  group_count: 8
  group_validator_count: 100
  group_local_aggregator_count: 10
  global_aggregator_count: 10

# Gossipsub Network Configuration
gossip:
  mesh_n: 6
  non_mesh_n: 4

# Cryptographic Constants
consts:
  signature_time: 20ms
  signature_size: {sig_size}
  snark_size: 131072
  snark1_threshold: 0.75
  snark2_threshold: 0.66
  aggregation_rate_per_sec: 1000
  snark_recursion_aggregation_rate_per_sec: 50
  pq_signature_verification_time: 30us
  snark_proof_verification_time: 5ms

# Network Simulation Parameters
network:
  gml: "shadow-atlas.bin"
""")
            
            # Run simulation with modified config
            run_kwargs = {**base_run_kwargs, 'c': modified_yaml}
            items = beamsim.run(**run_kwargs, t=topology, local_aggregation_only=True)
            
            # Extract SNARK1 timing information
            snark1_sent = beamsim.get_snark1_sent(items)
            if snark1_sent and len(snark1_sent[0]) > 0:
                # Get the time when SNARK1 generation completed
                snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
                
                # Get simulation info
                _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, "info")[0]
                
                results.append({
                    'topology': topology,
                    'signature_size': sig_size,
                    'snark1_completion_time': snark1_completion_time,
                    'validator_count': validator_count,
                    'snark1_threshold': snark1_threshold,
                    'topology_display': beamsim.topology_name[topology]
                })
    
    # Convert results to DataFrame for easier plotting
    df = pd.DataFrame(results)
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Plot lines for each topology
    colors = sns.color_palette("husl", len(topologies))
    
    for i, topology in enumerate(topologies):
        topo_data = df[df['topology'] == topology]
        plt.plot(
            topo_data['signature_size'] / 1024,  # Convert to KB for readability
            topo_data['snark1_completion_time'],
            marker='o',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=f'{beamsim.topology_name[topology]} Topology',
            alpha=0.8
        )
    
    # Enhance the plot
    plt.xlabel('Signature Size (KB)', fontweight='bold', fontsize=12)
    plt.ylabel('SNARK1 Aggregation Time (ms)', fontweight='bold', fontsize=12)
    plt.title('Effect of Signature Size on SNARK1 Aggregation Time\n(Local Aggregation Only Mode)', 
              fontweight='bold', fontsize=14)
    
    plt.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Add annotations showing the signature size effect
    if len(df) > 0:
        min_time = df['snark1_completion_time'].min()
        max_time = df['snark1_completion_time'].max()
        improvement = ((max_time - min_time) / max_time) * 100
        
        plt.text(0.02, 0.98, 
                f'Max time difference: {max_time - min_time:.1f}ms\n'
                f'Relative improvement: {improvement:.1f}%',
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    for topology in topologies:
        topo_data = df[df['topology'] == topology]
        if len(topo_data) > 0:
            print(f"\n{beamsim.topology_name[topology]} Topology:")
            print(f"  Signature size range: {topo_data['signature_size'].min()} - {topo_data['signature_size'].max()} bytes")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  Time difference: {topo_data['snark1_completion_time'].max() - topo_data['snark1_completion_time'].min():.1f} ms")
    
    return df

In [ ]:
def plot_bandwidth_effect(bandwidth_limits, signature_size=3072, topologies=["gossip", "grid"], **base_run_kwargs):
    """
    Plot how changing max incoming bandwidth affects SNARK1 aggregation time across different topologies.
    
    Args:
        bandwidth_limits: List of bandwidth limits to test (in Mbps, None for unlimited)
        signature_size: Fixed signature size to use for all tests (in bytes)
        topologies: List of network topologies to test
        **base_run_kwargs: Base simulation parameters
    """
    results = []
    
    print(f"Testing bandwidth limits: {bandwidth_limits}")
    print(f"Fixed signature size: {signature_size} bytes")
    print(f"Topologies: {topologies}")
    print("Running simulations...\n")
    
    for topology in topologies:
        print(f"Testing topology: {topology}")
        
        for bandwidth in bandwidth_limits:
            bandwidth_display = f"{bandwidth} Mbps" if bandwidth is not None else "Unlimited"
            print(f"  Max incoming bandwidth: {bandwidth_display}")
            
            # Create a modified YAML config with the new bandwidth limit
            bandwidth_config = ""
            if bandwidth is not None:
                bandwidth_config = f"  max_incoming_bandwidth: {bandwidth}Mbps"
            
            modified_yaml = beamsim.yaml(f"""
# Simulation Backend Configuration
backend: ns3-direct
                                        
snark1_pull: true
snark1_half_direct: true

# Network Topology Configuration
topology: {topology}

# Whether to shuffle validators from the same group to different routers
shuffle: false

# Seed for reproducible simulation results
random_seed: 42

# Role Assignment Configuration
roles:
  group_count: 8
  group_validator_count: 100
  group_local_aggregator_count: 10
  global_aggregator_count: 10

# Gossipsub Network Configuration
gossip:
  mesh_n: 6
  non_mesh_n: 4

# Cryptographic Constants
consts:
  signature_time: 20ms
  signature_size: {signature_size}
  snark_size: 131072
  snark1_threshold: 0.75
  snark2_threshold: 0.66
  aggregation_rate_per_sec: 1000
  snark_recursion_aggregation_rate_per_sec: 50
  pq_signature_verification_time: 30us
  snark_proof_verification_time: 5ms

# Network Simulation Parameters
network:
  gml: "shadow-atlas.bin"
{bandwidth_config}
""")
            
            # Run simulation with modified config
            run_kwargs = {**base_run_kwargs, 'c': modified_yaml}
            items = beamsim.run(**run_kwargs, t=topology, local_aggregation_only=True)
            
            # Extract SNARK1 timing information
            snark1_sent = beamsim.get_snark1_sent(items)
            if snark1_sent and len(snark1_sent[0]) > 0:
                # Get the time when SNARK1 generation completed
                snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
                
                # Get simulation info
                _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, "info")[0]
                
                results.append({
                    'topology': topology,
                    'bandwidth_limit': bandwidth,
                    'bandwidth_display': bandwidth_display,
                    'snark1_completion_time': snark1_completion_time,
                    'validator_count': validator_count,
                    'snark1_threshold': snark1_threshold,
                    'topology_display': beamsim.topology_name[topology]
                })
    
    # Convert results to DataFrame for easier plotting
    df = pd.DataFrame(results)
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Plot lines for each topology
    colors = sns.color_palette("husl", len(topologies))
    
    for i, topology in enumerate(topologies):
        topo_data = df[df['topology'] == topology]
        
        # Create proper x-axis positions
        x_values = []
        y_values = []
        
        # First, handle non-None bandwidth values in order
        non_none_data = topo_data[topo_data['bandwidth_limit'].notna()].copy()
        non_none_data = non_none_data.sort_values('bandwidth_limit')
        
        for idx, (_, row) in enumerate(non_none_data.iterrows()):
            x_values.append(idx)
            y_values.append(row['snark1_completion_time'])
        
        # Then handle None (unlimited) bandwidth
        none_data = topo_data[topo_data['bandwidth_limit'].isna()]
        if len(none_data) > 0:
            x_values.append(len(non_none_data))  # Place unlimited at the end
            y_values.append(none_data.iloc[0]['snark1_completion_time'])
        
        plt.plot(
            x_values,
            y_values,
            marker='o',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=f'{beamsim.topology_name[topology]} Topology',
            alpha=0.8
        )
    
    # Enhance the plot
    plt.xlabel('Max Incoming Bandwidth', fontweight='bold', fontsize=12)
    plt.ylabel('SNARK1 Aggregation Time (ms)', fontweight='bold', fontsize=12)
    plt.title(f'Effect of Max Incoming Bandwidth on SNARK1 Aggregation Time\n(Signature Size: {signature_size/1024:.1f} KB, Local Aggregation Only)', 
              fontweight='bold', fontsize=14)
    
    # Set x-axis labels
    numeric_bandwidths = [bw for bw in bandwidth_limits if bw is not None]
    numeric_bandwidths.sort()
    x_labels = [f"{bw} Mbps" for bw in numeric_bandwidths]
    if None in bandwidth_limits:
        x_labels.append("Unlimited")
    
    plt.xticks(range(len(x_labels)), x_labels, rotation=45)
    
    plt.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Add annotations showing the bandwidth effect
    if len(df) > 0:
        min_time = df['snark1_completion_time'].min()
        max_time = df['snark1_completion_time'].max()
        improvement = ((max_time - min_time) / max_time) * 100
        
        plt.text(0.02, 0.98, 
                f'Max time difference: {max_time - min_time:.1f}ms\n'
                f'Performance improvement: {improvement:.1f}%',
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("BANDWIDTH EFFECT SUMMARY")
    print("="*60)
    for topology in topologies:
        topo_data = df[df['topology'] == topology]
        if len(topo_data) > 0:
            print(f"\n{beamsim.topology_name[topology]} Topology:")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  Time difference: {topo_data['snark1_completion_time'].max() - topo_data['snark1_completion_time'].min():.1f} ms")
            
            # Show performance for each bandwidth setting
            for _, row in topo_data.iterrows():
                print(f"    {row['bandwidth_display']:>12}: {row['snark1_completion_time']:.1f} ms")
    
    return df

In [ ]:
# Default YAML Configuration for Local Aggregation Only Simulations
# This configuration is specifically tuned for local aggregation only mode
# The simulation stops after local aggregators generate SNARK1

yaml_config_path = beamsim.yaml("""
# Simulation Backend Configuration. Options: delay, queue, ns3-direct
# - delay: Simple delay-based network simulation
# - queue: Queue-based network simulation with more realistic modeling
# - ns3-direct: NS-3 simulation with direct peer connections
backend: ns3-direct
                                
snark1_pull: true
snark1_half_direct: true

# Network Topology Configuration. Options: direct, gossip, grid
topology: gossip

# Whether to shuffle validators from the same group to different routers
shuffle: false

# Seed for reproducible simulation results
random_seed: 42

# Role Assignment Configuration
# Note: Global aggregators are less relevant in local aggregation only mode
roles:
  # Number of validator groups (affects parallel processing and aggregation)
  group_count: 8
  
  # Number of validators per group (including local aggregators)
  group_validator_count: 100
  
  # Number of local aggregators per group (more important in this mode)
  group_local_aggregator_count: 10

  # Total number of global aggregators (not used in local aggregation only)
  global_aggregator_count: 10

# Gossipsub Network Configuration (only applies when topology=gossip)
gossip:
  # Target number of peers in the mesh network for each topic
  mesh_n: 6
  
  # Number of non-mesh peers to maintain connections with
  non_mesh_n: 4

# Cryptographic Constants
consts:
  # Time for each validator to generate their initial signature
  signature_time: 20ms
  
  # Size of individual signatures in bytes (affects network traffic)
  signature_size: 3072
  
  # Size of SNARK proofs in bytes (significantly affects bandwidth)
  snark_size: 131072
  
  # Threshold fraction of signatures needed for SNARK1 generation (0.0-1.0)
  snark1_threshold: 0.75
  
  # Threshold fraction of signatures needed for final SNARK2 (not used in local aggregation only)
  snark2_threshold: 0.66
  
  # Rate of signature aggregation (signatures per second)
  aggregation_rate_per_sec: 1000
  
  # Rate of SNARK proof recursion/aggregation (proofs per second)
  snark_recursion_aggregation_rate_per_sec: 50
  
  # Time to verify a single post-quantum signature
  pq_signature_verification_time: 30us
  
  # Time to verify a single SNARK proof
  snark_proof_verification_time: 5ms

# Network Simulation Parameters
network:
  # Take latencies from shadow atlas file
  gml: "shadow-atlas.bin"
""")

In [ ]:
# Run simulations using YAML config as defaults with LOCAL AGGREGATION ONLY mode
# CLI flags can override the YAML configuration values
# The simulator binary loads YAML config first, then applies CLI flag overrides
# The --local-aggregation-only flag stops simulation after local aggregators generate SNARK1

# Optional CLI flag overrides (uncomment and modify as needed)
run_kwargs = dict(
    c=yaml_config_path,
    # Uncomment any of these to override the YAML config defaults:
    # b="ns3-direct",  # backend override
    # g=10,  # groups override
    # gv=128,  # group validators override
    mpi=9,  # enable MPI: set to False to disable, alternatively set to number of processes
    # la=1,  # local aggregators override
    # ga=1,  # global aggregators override (not used in local aggregation only)
    # shuffle=False,  # shuffle override
    # Note: local_aggregation_only=True is set automatically in the plotting functions
)

topologies = ["gossip", "grid"]

# Test different signature sizes to analyze their effect on SNARK1 aggregation time
# Signature sizes in bytes - testing a range from small to large signatures

signature_sizes = [
    1024,    # 1 KB - small signature
    2048,    # 2 KB - medium-small signature  
    3072,    # 3 KB - default signature size
    4096,    # 4 KB - medium signature
    6144,    # 6 KB - large signature
    8192,    # 8 KB - very large signature
]

# Base simulation parameters (without signature size)
base_run_kwargs = dict(
    # Use MPI for faster simulations
    mpi=4,
    # Keep other parameters from original config
    # The signature size will be set dynamically in the plotting function
)

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of signature size on SNARK1 aggregation time...")
print("This will run simulations with different signature sizes and compare results.\n")

# Run the signature size effect analysis
results_df = plot_signature_size_effect(signature_sizes, topologies, **base_run_kwargs)

In [ ]:
# Analyze the effect of max incoming bandwidth on SNARK1 aggregation time
# Test different bandwidth limits to understand network bottlenecks

bandwidth_limits = [
    1,      # 1 Mbps - Very constrained
    5,      # 5 Mbps - Low bandwidth
    10,     # 10 Mbps - Medium-low bandwidth
    50,     # 50 Mbps - Medium bandwidth
    100,    # 100 Mbps - High bandwidth
    500,    # 500 Mbps - Very high bandwidth
    None    # Unlimited bandwidth (no constraint)
]

# Use fixed signature size of 3072 bytes (3KB) as requested
signature_size = 3072

# Base simulation parameters for bandwidth analysis
bandwidth_run_kwargs = dict(
    mpi=4,  # Use MPI for faster simulations
)

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of max incoming bandwidth on SNARK1 aggregation time...")
print(f"Using fixed signature size: {signature_size} bytes ({signature_size/1024:.1f} KB)")
print("This will help identify network bandwidth bottlenecks in signature collection.\n")

# Run the bandwidth effect analysis
bandwidth_results_df = plot_bandwidth_effect(bandwidth_limits, signature_size, topologies, **bandwidth_run_kwargs)